# Step 1: Generate The Similar Dictionary Map

Download the Glove from https://github.com/stanfordnlp/GloVe

In [ ]:
# obtain the similar words library from https://github.com/stanfordnlp/GloVe
!wget https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
!unzip -p glove.6B.zip glove.6B.300d.txt > glove.6B.300d.txt

--2022-10-10 05:07:45--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2022-10-10 05:07:45--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.00MB/s    in 2m 39s  

2022-10-10 05:10:24 (5.18 MB/s) - ‘glove.6B.zip.1’ saved [862182753/862182753]



Start Generate the similar words dictionary map

In [ ]:
import numpy as np
import sys
import pickle

def generateSimilarDict():
  print("Reading similar dictionary")
  with open("glove.6B.300d.txt", 'r') as f:
    words = [x.rstrip().split(' ')[0] for x in f.readlines()]

  print("Reading similar dictionary Vectors")
  with open("glove.6B.300d.txt", 'r') as f:
    vectors = {}
    for line in f:
        vals = line.rstrip().split(' ')
        vectors[vals[0]] = [float(x) for x in vals[1:]]

  vocab_size = len(words)
  vocab = {w: idx for idx, w in enumerate(words)}
  ivocab = {idx: w for idx, w in enumerate(words)}

  vector_dim = len(vectors[ivocab[0]])
  W = np.zeros((vocab_size, vector_dim))
  for word, v in vectors.items():
      if word == '<unk>':
          continue
      W[vocab[word], :] = v

  # normalize each word vector to unit variance
  W_norm = np.zeros(W.shape)
  d = (np.sum(W ** 2, 1) ** (0.5))
  W_norm = (W.T / d).T

  print("Save Dictionary")
  with open('similar_dict_glove.6B.300d.pickle', 'wb') as handle:
    pickle.dump((W_norm, vocab, ivocab), handle, protocol=pickle.HIGHEST_PROTOCOL)

generateSimilarDict()

Reading similar dictionary
Reading similar dictionary Vectors


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


Save Dictionary


#Step 2: Generate Symbolic List

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!wget https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/SimilarDict.py
!wget https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/data/categories_places365.txt
!wget https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/data/labels_sunattribute.txt
!wget https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/Places365.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ebr_c3p0
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ebr_c3p0
--2022-10-10 05:12:14--  https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/SimilarDict.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1454 (1.4K) [text/plain]
Saving to: ‘SimilarDict.py.1’

SimilarDict.py.1    100%[===================>]   1.42K  --.-KB/s    in 0s      

2022-10-10 05:12:14 (20.3 MB/s) - ‘SimilarDict.py.1’ saved [1454/1454]

--2022-10-10 05:12:14--  https://raw.githubusercontent.com/CornerSiow/zero-shot-image-caption

In [ ]:
from SimilarDict import SimilarDict
from Places365 import Places365
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from nltk.tokenize import word_tokenize, MWETokenizer
import os
from tqdm import tqdm

In [ ]:
def load_coco_labels():
    things = []
    stuff = []
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    meta = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
    for v in meta.thing_classes:
        things.append(v.replace("-", " ").replace("_", " "))
       
    for v in meta.stuff_classes:
        stuff.append(v.replace("-", " ").replace("_", " "))
       
    return things, stuff


In [ ]:
def load_palces365_labels():
    # prepare all the labels
    # scene category relevant
    file_name_category = 'categories_places365.txt'
    if not os.access(file_name_category, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name_category) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = list(classes)
    for i, v in enumerate(classes):
        classes[i] = v.replace("_", " ").replace("/", " ")
   
    # scene attribute relevant
    file_name_attribute = 'labels_sunattribute.txt'
    if not os.access(file_name_attribute, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/labels_sunattribute.txt'
        os.system('wget ' + synset_url)
    with open(file_name_attribute) as f:
        lines = f.readlines()
        labels_attribute = [item.rstrip() for item in lines]
    
    for i, v in enumerate(labels_attribute):
        labels_attribute[i] = v.replace("-", " ")

    return classes, labels_attribute


In [ ]:
similarDict = SimilarDict(0.69)
place365 = Places365()
places365Class, places365Attribute = load_palces365_labels()
things, stuff = load_coco_labels()

In [ ]:
symbolic_list = things + stuff + places365Class + places365Attribute
print("All available symbolic: {:d}".format(len(symbolic_list)))
print("Remove repeated symbolic...")
symbolic_list = list(set(symbolic_list))
for i, v in enumerate(symbolic_list):
    symbolic_list[i] = v.replace(" ", "_")
print("All unique symbolic: {:d}".format(len(symbolic_list)))

All available symbolic: 601
Remove repeated symbolic...
All unique symbolic: 587


In [ ]:
#finding all similar words:
print("Finding similar symbolic words...")
similar_symbolic = {}
similarDict = SimilarDict(threshold=0.69)
for symbolic in tqdm(symbolic_list):
    # Get the unique symbolic index
    symbolic_index = symbolic_list.index(symbolic)
    
    # Check is unique symbolic is in the similar list?
    if symbolic not in similar_symbolic:
        similar_symbolic[symbolic] = []
    
    # append the symbolic index to the similar word
    if symbolic_index not in similar_symbolic[symbolic]:
        similar_symbolic[symbolic].append(symbolic_index)
   
    result = similarDict(symbolic.replace("_", " "))  
    for r in result:
        word = r[0]
        if word not in similar_symbolic:
            similar_symbolic[word] = []
            
        # append the symbolic index to the similar word
        if symbolic_index not in similar_symbolic[word]:
            similar_symbolic[word].append(symbolic_index)
    


print("All similar symbolic: {:d}".format(len(similar_symbolic)))

Finding similar symbolic words...


100%|██████████| 587/587 [03:45<00:00,  2.60it/s]

All similar symbolic: 836


In [ ]:
print("Save Symbolic List...")
with open('symbolic_list.pickle', 'wb') as handle:
    pickle.dump(symbolic_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('similar_symbolic.pickle', 'wb') as handle:
    pickle.dump(similar_symbolic, handle, protocol=pickle.HIGHEST_PROTOCOL)

Save Symbolic List...


# Step 3: Generate Vocabulary

In [ ]:
!wget https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/Vocabulary.py
!wget https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/data/caption_label.txt
!pip install nltk

--2022-10-10 05:33:46--  https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/Vocabulary.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3150 (3.1K) [text/plain]
Saving to: ‘Vocabulary.py.3’

Vocabulary.py.3     100%[===================>]   3.08K  --.-KB/s    in 0s      

2022-10-10 05:33:46 (38.9 MB/s) - ‘Vocabulary.py.3’ saved [3150/3150]

--2022-10-10 05:33:46--  https://raw.githubusercontent.com/CornerSiow/zero-shot-image-captioning/main/data/caption_label.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


In [ ]:
import nltk
from Vocabulary import Vocabulary
nltk.download('punkt')
vocab = Vocabulary()
vocab.loadCaptions("caption_label.txt")
vocab.saveFile('vocab.pickle')
print("Save Vocab...")

Save Vocab...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Step 4: Remove Non-related Symbolics

In [ ]:
print("Unique Symbolic: {:d}".format(len(symbolic_list)))
print("Similar Symbolic: {:d}".format(len(similar_symbolic)))
tokenize = MWETokenizer()
for v in symbolic_list:    
    tokenize.add_mwe(v.split("_"))

Unique Symbolic: 587
Similar Symbolic: 836


In [ ]:
print("Read Training Caption...")
print("Start filtering unrelated symbolic...")
annFile = "caption_label.txt"
filtered_symbolic = []
with open(annFile) as file:
    for line in file:
        cap = line.strip().lower()               
        tokens = tokenize.tokenize(word_tokenize(cap)) 
        activated = []
        for w in tokens:            
            if w in similar_symbolic:
                for i in similar_symbolic[w]:
                    activated.append(symbolic_list[i])
        
        for w in activated:
          if w not in filtered_symbolic:
            filtered_symbolic.append(w)    

Read Training Caption...
Start filtering unrelated symbolic...


In [ ]:
print("Filtered Result...")
print(filtered_symbolic)
print("Filtered Symbolic: {:d}".format(len(filtered_symbolic)))

with open('filtered_symbolic.pickle', 'wb') as handle:
    pickle.dump(filtered_symbolic, handle, protocol=pickle.HIGHEST_PROTOCOL)

Filtered Result...
['person', 'laptop', 'office', 'banana', 'sink', 'bicycle', 'sky', 'bus']
Filtered Symbolic: 8


#Step 5: Generate Training Data

In [79]:
import torch
print("Start Generate Training Data...")
annFile = "caption_label.txt"
data = []
with open(annFile) as file:
    for line in file:
        cap = line.strip().lower()       
        feature = np.zeros(len(filtered_symbolic))
        tokens = tokenize.tokenize(word_tokenize(cap)) 
        activated_symbolic = []
        for w in tokens:            
            if w in similar_symbolic:
                for i in similar_symbolic[w]:
                    # if symbolic_list[i] not in ['sky','waiting_in_line','motorcycle','biking', 'man_made', 'truck','train','train_station_platform']:
                        activated_symbolic.append(symbolic_list[i])
           
        
        
        for symbolic in activated_symbolic:
            if symbolic in filtered_symbolic:               
                feature[filtered_symbolic.index(symbolic)] = 1
       
        # feature = feature / np.max(feature)
        
        print('----------------------------------')
        print(cap, '\n',activated_symbolic,'\n', feature)        
                
        data.append((torch.from_numpy(feature), vocab.convertSentenceToToken(cap)))
print('----------------------------------')
print("Finish Generate Training Data...")        
print("Save Training Data...")
with open('training_data.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)


Start Generate Training Data...
----------------------------------
a person using a laptop in the office 
 ['person', 'laptop', 'office'] 
 [1. 1. 1. 0. 0. 0. 0. 0.]
----------------------------------
a person eats a banana in front of a laptop 
 ['person', 'banana', 'laptop'] 
 [1. 1. 0. 1. 0. 0. 0. 0.]
----------------------------------
a person washes his face in the sink 
 ['person', 'sink'] 
 [1. 0. 0. 0. 1. 0. 0. 0.]
----------------------------------
a person riding a bike on a clear sky 
 ['person', 'bicycle', 'sky'] 
 [1. 0. 0. 0. 0. 1. 1. 0.]
----------------------------------
someone is waiting at the bus stop 
 ['person', 'bus'] 
 [1. 0. 0. 0. 0. 0. 0. 1.]
----------------------------------
Finish Generate Training Data...
Save Training Data...
